# Exploratory data analysis

**EDA using local data from the library of Aarhus and local weather data**

The data used in this project is fetched from API's provided by local authorities.

**Disclaimer: The sensors used at the library are rather inconsistent, which can generate some strange output **

In [56]:
import pandas as pd
import numpy as np
import matplotlib as plt
from matplotlib import pyplot

## Understanding library traffic during a pandemic

![dokk1](https://upload.wikimedia.org/wikipedia/commons/thumb/5/51/Dokk1_version_3.jpg/2560px-Dokk1_version_3.jpg)

### Case
The data used in this section is fetched from an API containing live data from my home city Aarhus
Live data source (URL #1): https://admin.opendata.dk/datastore/dump/68bf80d4-4910-49b6-b5c5-19e41726cc81

*The data is updated every 5 minutes.*

Further documentation is provided [here](https://www.opendata.dk/city-of-aarhus/besogstal-og-abningstider-for-aarhus-kommunes-biblioteker#resource-biblioteksnumre). The dataset with names of libraries and library numbers can be fetched from here (URL #2): https://portal.opendata.dk/dataset/f5acc4f7-985a-4284-898b-30d34638d445/resource/02acfa44-2a6b-4dd8-a433-ad1f16604395/download/biblioteksnumre.csv

#### Import traffic data from API

In [20]:
#1 traffic data
df_visitors = pd.read_csv('https://admin.opendata.dk/datastore/dump/68bf80d4-4910-49b6-b5c5-19e41726cc81')
print(df_visitors.columns)

#2 library name data 
df_names = pd.read_csv('https://portal.opendata.dk/dataset/f5acc4f7-985a-4284-898b-30d34638d445/resource/02acfa44-2a6b-4dd8-a433-ad1f16604395/download/biblioteksnumre.csv',sep='\t')
print(df_names.columns)

Index(['_id', 'libno', 'time', 'in', 'out'], dtype='object')
Index(['libno', 'branch'], dtype='object')


#### About the data:
Traffick data:
- `_id`: ID as provided by DOKK1.
- `libno`: Library number.
- `in`: Amount of people observed gone inside today.
- `out`: Amount of people observed gone outside today.
- `time`: Time of record.

Name data:
- `libno`: Library number..
- `branch`: Library name.

### Data cleaning
The objective is to tidy the library names in order to reach naming consistancy

In [21]:
#The data before cleaning:
df_names

,libno,branch
0,775122,Beder-Malling Bibliotek
1,775164,Egå Kombi-bibliotek
2,775144,Gellerup Bibliotek
3,775146,Harlev Bibliotek
4,775147,Hasle Bibliotek
5,775162,Hjortshøj Kombi-bibliotek
6,775100,Hovedbiblioteket
7,775120,Højbjerg Bibliotek
8,775133,Kolt-Hasselager Kombi-bibliotek
9,775167,Lystrup Bibliotek


In [23]:
df_names['branch'] = df_names['branch'].str.capitalize() # Make first letter of the column uppercase
df_names['branch'] = df_names['branch'].str.replace('- komforbi','') # remove '- komforbi
df_names['branch'] = df_names['branch'].str.replace('kombi-','') # remove kombi-
df_names

# Can argue that hovedbiblioteket can be abit more tidy - maybe implement a " " and change end to bibliotek

,libno,branch
0,775122,Beder-malling bibliotek
1,775164,Egå bibliotek
2,775144,Gellerup bibliotek
3,775146,Harlev bibliotek
4,775147,Hasle bibliotek
5,775162,Hjortshøj bibliotek
6,775100,Hovedbiblioteket
7,775120,Højbjerg bibliotek
8,775133,Kolt-hasselager bibliotek
9,775167,Lystrup bibliotek


#### Join the dataframes on libno

In [6]:
df = pd.merge(df_visitors, df_names, on='libno') # Specifying to joing on 'libno' since this is the common key in both DF's 
df.head()

,_id,libno,time,in,out,branch
0,1,775146,2021-01-08T17:50:18,0,0,Harlev bibliotek
1,2,775167,2021-01-08T17:50:18,0,0,Lystrup bibliotek
2,3,775168,2021-01-08T17:50:18,0,0,Skødstrup bibliotek
3,4,775127,2021-01-08T17:50:18,0,0,Tranbjerg bibliotek
4,5,775122,2021-01-08T17:50:18,0,0,Beder-malling bibliotek


#### Using f-strings to print the sum of people in the libraries on the current day and time of running the script

In [7]:
from datetime import datetime
from datetime import date

a_visitors = df['in'].sub(df['out']).sum()

df['time'] = pd.to_datetime(df['time']) # time column as datetime

hoved = df.loc[17, 'branch'] # locate and extract hovedbiblioteket for future reference

now = datetime.now() # create an object with a current time
now = now.strftime("%H:%M") # shortening it down to only hours and minutes

today = date.today() # Create an object with the current date
today = today.strftime("%B %d") # shortening it down written month and date

print(f"There were {a_visitors} people inside {hoved} (DOKK1) on {today} at {now}.")

# This exercise could also be solved by simply locating "hovedbibliotektet" and refer to columns of that row with
#.iloc/.loc or plain python indexing

There were -1 people inside Hovedbiblioteket (DOKK1) on January 08 at 17:51.


#### Printing todays most popular library based on how many people entered the library

In [8]:
df = df.sort_values('in', ascending=False)
print(f"The popular library today is: {df.iloc[0,5]} (see more info below)")
df[0:1]

The popular library today is: Hovedbiblioteket (see more info below)


,_id,libno,time,in,out,branch
17,18,775100,2021-01-08 17:50:18,63,57,Hovedbiblioteket


#### Show names of libraries without any visitors today.

In [9]:
df[(df['in'] == 0) & (df['out'] == 0)]
#see next cell for another way of showing this with eq.(0)

,_id,libno,time,in,out,branch
10,11,775149,2021-01-08 17:50:18,0,0,Sabro bibliotek
15,16,775150,2021-01-08 17:50:18,0,0,Tilst bibliotek
12,13,775162,2021-01-08 17:50:18,0,0,Hjortshøj bibliotek
0,1,775146,2021-01-08 17:50:18,0,0,Harlev bibliotek
1,2,775167,2021-01-08 17:50:18,0,0,Lystrup bibliotek
7,8,775120,2021-01-08 17:50:18,0,0,Højbjerg bibliotek
6,7,775126,2021-01-08 17:50:18,0,0,Solbjerg bibliotek
5,6,775140,2021-01-08 17:50:18,0,0,Åby bibliotek
4,5,775122,2021-01-08 17:50:18,0,0,Beder-malling bibliotek
3,4,775127,2021-01-08 17:50:18,0,0,Tranbjerg bibliotek


#### Adding a new column with the ratio of people left the library i.e. procentage of people who left the library

In [17]:
df['inside_estimate'] = (df['out'].values / df['in'].values)*100 # Create the column showing decimal
df

C:\Users\siras\Anaconda3\envs\idsp\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.
C:\Users\siras\Anaconda3\envs\idsp\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


,_id,libno,time,in,out,branch,inside_estimate
17,18,775100,2021-01-08 17:50:18,63,57,Hovedbiblioteket,90.476190
16,17,775147,2021-01-08 17:50:18,32,26,Hasle bibliotek,81.250000
14,15,775160,2021-01-08 17:50:18,8,7,Risskov bibliotek,87.500000
13,14,775144,2021-01-08 17:50:18,7,18,Gellerup bibliotek,257.142857
8,9,775130,2021-01-08 17:50:18,2,4,Viby bibliotek,200.000000
10,11,775149,2021-01-08 17:50:18,0,0,Sabro bibliotek,NaN
15,16,775150,2021-01-08 17:50:18,0,0,Tilst bibliotek,NaN
12,13,775162,2021-01-08 17:50:18,0,0,Hjortshøj bibliotek,NaN
11,12,775170,2021-01-08 17:50:18,0,1,Trige bibliotek,inf
0,1,775146,2021-01-08 17:50:18,0,0,Harlev bibliotek,NaN


## Temperature in Aarhus

![image](https://f.nordiskemedier.dk/2f94zq7shph0o0gb_1920_640_c.jpg)

Let's have a look at some temperature data measured between DOKK1 and Navitas from before 2018.

### Load weather data & data exploration

In [48]:
# This is loaded in without using the time as an index - this will be experimented with in the last task
data = pd.read_csv('Desktop/Uni/Python/assignment_a-updated/tempdokk1nav.csv')  
data['time'] = pd.to_datetime(data['time']) # Update the entire time column to datetime64

print('Dimensions:', data.shape,'\n')
data.head()

Dimensions: (26387, 2) 



,time,val
0,2016-08-18 10:55:00,25.81
1,2016-08-18 11:00:00,25.74
2,2016-08-18 11:05:00,25.81
3,2016-08-18 11:10:00,25.81
4,2016-08-18 11:15:00,25.86


#### The timeperiod recorded in the dataset

In [49]:
print(f"The timestamp of the first observation is: {data['time'].min()}")
print(f"The timestamp of the last observation is: {data['time'].max()}")

The timestamp of the first observation is: 2016-08-18 10:55:00
The timestamp of the last observation is: 2017-12-18 08:25:00


#### Looking at the avgerage temperature in July only including daytime temparatures (from 8:00-16:00)

In [72]:
# Defining the "time mask" and applying it to the DF, thus filtering away all observations that isn't in the timeframe.
id_df_interval = id_df.between_time('08:00','16:00') 

# Overwriting the dataframe to only include the 7th month (july) with
id_df_interval = id_df_interval[id_df_interval.index.month == 7]

# Taking the mean of the values left in the DF (time bewteen 8 and 16 in july 2017)
mean_temp = id_df_interval.mean() 

print(f"The mean temperature of july in the daytime is= {mean_temp.values.round(1)}")

The mean temperature of july in the daytime is= [20.8]
